<a href="https://colab.research.google.com/github/dimsparagis0210/DFU-using-Deep-Learning/blob/main/EfficientNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EfficientNet Model Development

#### The goal of this BSc thesis is to develop a Deep Learning model for Object Detection. Specifically, the object of the model will be to detect ulcers in Diabetic Foots (DFU)

## Getting Dataset from Google drive

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

# Define dataset path (adjust if needed)
dataset = '/content/drive/MyDrive/DFU_Data_Tensorflow'
original = dataset + '/images'
trainset = dataset + '/train'
testset = dataset + '/test'
validset = dataset + '/valid'

# List categories (folders)
os.listdir('/content')

['.config', 'drive', 'my_dir', 'sample_data']

In [ ]:
import tensorflow as tf
from tensorflow import keras
import os
import cv2
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

## Data Preprocessing

#### The dataset contains a csv with the annotations' coordinates

In [ ]:
import pandas as pd

# Load the CSV file
annotations = pd.read_csv(trainset + '/_annotations.csv')

# Display the first few rows to verify
print(annotations.head())

                                            filename  width  height  class  \
0  51814847_548907345629031_1716986440128135168_n...    640     640  Ulcer   
1  51814847_548907345629031_1716986440128135168_n...    640     640  Ulcer   
2  51814847_548907345629031_1716986440128135168_n...    640     640  Ulcer   
3     20_jpg.rf.2ae76d51fdf025da6450aaa95081e571.jpg    640     640     bb   
4  40152324_676265276071443_3891772576753516544_n...    640     640  Ulcer   

   xmin  ymin  xmax  ymax  
0    88   115   237   321  
1   382   157   523   407  
2   594   222   640   425  
3   255   190   387   269  
4   147   356   235   392  


In [ ]:
# Get unique class names
class_names = annotations['class'].unique()

# Create a mapping from class names to integers
class_mapping = {name: index for index, name in enumerate(class_names)}

# Apply the mapping to the 'class' column
annotations['class'] = annotations['class'].map(class_mapping)


In [ ]:
def load_and_preprocess_image(image_path, target_size=(224, 224)):
    # Read the image file
    image = tf.io.read_file(image_path)
    # Decode the image
    image = tf.image.decode_jpeg(image, channels=3)
    # Resize the image
    image = tf.image.resize(image, target_size)
    # Normalize pixel values to [0, 1]
    image = image / 255.0
    return image


In [ ]:
def parse_function(filename, bbox, label, image_dir):
    # Construct the full path to the image file using TensorFlow operations
    image_path = tf.strings.join([image_dir, filename], separator=os.sep)
    # Load and preprocess the image
    image = load_and_preprocess_image(image_path)
    # Normalize bounding box coordinates to [0, 1]
    bbox = tf.stack(bbox)
    return image, {'bbox': bbox, 'label': label}


def create_dataset(annotations, image_dir, batch_size=32):
    # Extract columns from the DataFrame
    filenames = annotations['filename'].values
    bboxes = annotations[['xmin', 'ymin', 'xmax', 'ymax']].values
    labels = annotations['class'].values
    # Create a TensorFlow Dataset from the filenames, bounding boxes, and labels
    dataset = tf.data.Dataset.from_tensor_slices((filenames, bboxes, labels))
    # Map the parse function over the dataset
    dataset = dataset.map(lambda f, b, l: parse_function(f, b, l, image_dir))
    # Shuffle, batch, and prefetch the dataset
    dataset = dataset.shuffle(buffer_size=len(filenames))
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    return dataset

# Directory containing the images
image_dir_train = trainset
image_dir_valid = validset

# Create the dataset
train_dataset = create_dataset(annotations, image_dir_train)
valid_dataset = create_dataset(annotations, image_dir_valid)

### Data Augmentation Layer

In [ ]:
from tensorflow.keras import layers

data_augmentation = keras.models.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.1),
    layers.RandomBrightness(0.1)
])

### Apply the preprocessing steps to the train and validation set

In [ ]:
def preprocess_train(image, label):
    image = data_augmentation(image)
    return image, label

train_dataset = train_dataset.map(preprocess_train, num_parallel_calls=tf.data.AUTOTUNE)

## Model Training

### EfficientNet

#### Hyperparameter Optimization

In [ ]:
pip install keras-tuner

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner as kt

def build_model(hp):
    inputs = keras.Input(shape=(224, 224, 3))
    # Load EfficientNetB0 with ImageNet weights and exclude the top layers
    base_model = keras.applications.EfficientNetB0(include_top=False, input_tensor=inputs, weights='imagenet')
    base_model.trainable = False  # Freeze the base model

    # Add global pooling layer
    x = layers.GlobalAveragePooling2D()(base_model.output)
    # Tune the number of units in the dense layer
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    x = layers.Dense(units=hp_units, activation='relu')(x)
    # Output layer
    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = keras.Model(inputs, outputs)

    # Tune the learning rate for the optimizer
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model


In [ ]:
from keras_tuner import RandomSearch

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    directory='my_dir',
    project_name='efficientnet_tuning'
)


In [ ]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

# Expand dimensions to match the input shape of the model
img_train = tf.expand_dims(img_train, axis=-1)
img_test = tf.expand_dims(img_test, axis=-1)

# Assuming img_train is your training dataset with shape (num_samples, 28, 28)
# Expand dimensions to add the channel axis

print(img_train.shape)

(60000, 28, 28, 1)


In [ ]:
# Resize images to 224x224
img_train = tf.image.resize(img_train, [224, 224])  # Shape becomes (num_samples, 224, 224, 1)

In [ ]:
# If your model expects 3 channels, you can replicate the single channel
img_train = tf.image.grayscale_to_rgb(img_train)  # Shape becomes (num_samples, 224, 224, 3)

ResourceExhaustedError: {{function_node __wrapped__Tile_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[60000,224,224,3] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Tile] name: 

In [ ]:
tuner.search(img_train, label_train, epochs=10, validation_split=0.2)

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
model = tuner.hypermodel.build(best_hps)